In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math

In [2]:
# Data loading
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
print(boston.data.shape)

(506, 13)


In [4]:
#Indicate function
def sign(x):
    cx = np.copy(x)
    length = cx.shape[0]
    signal = np.zeros(length)
    for i in range(length):
        if x[i,0] != 0:
            signal[i] = x[i,0] / abs(x[i,0])
        else:
            signal[i] = 0;
    return signal.reshape(length,1)

In [5]:
# Compare a value with each element in a vector
# and replace the element with the max one
def vec_max(value, vector):
    for i in range(len(vector)):
        vector[i] = max(value, vector[i,0])
    return vector

In [6]:
# f(x) = (1/2)||x'beta-y||^2
# Gradient of f(x) = 2*x'(x'beta - y)
def grf(A,x,y):
    return A.T.dot(A.dot(x) - y)

In [7]:
def lasso_proximal(X,y,lamb,MAX_ITER=2000):
    epsilon = 1e-4
    n = X.shape[1]
    beta = np.zeros(shape=(n,1))
    miu = np.linspace(0.0001,0.0001,n).reshape(1,n)
    cy = y.reshape(len(y),1)
    y = cy
    for i in range(MAX_ITER):
        beta_kplus1 = beta - miu.T*grf(X,beta,y)
        beta_kplus1 = vec_max(0, abs(beta_kplus1) - miu.T * lamb) * sign(beta_kplus1)
        if sum(abs(beta_kplus1 - beta)) < epsilon:
            beta = beta_kplus1
            break
        else:
            beta = beta_kplus1
#     print(i)
#     print(sum(abs(beta_kplus1 - beta)))
    return beta

In [8]:
def normalization_X(X):
    # Calculate mean  
    X_mean = X.mean(axis=0)  
    # Calculate variance   
    X_std = X.std(axis=0)  
    # Standardize X  
    X1 = (X-X_mean)/X_std
    return X1

def normalization_y(y):
    # Calculate mean  
    y_mean = y.mean  
    # Calculate variance   
    y_std = y.std  
    # Standardize y  
    y1 = (y-y_mean)/y_std
    return y1

In [9]:
# I use BIC as criterion
# BIC = n * ln(mse) + k * ln(n)
def get_BIC(y_pre,y_real,beta):
    mse = mean_squared_error(y_real,y_pre)
    n = len(y_real)
    k = sum(abs(sign(beta)))
    bic = n * math.log(mse) + k * np.log(n)
    return bic

In [10]:
# Get train and test data
X = boston.data
y = boston.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=19990226)

# Before using lasso, standardize the data
X_train_n = normalization_X(X_train)
X_test_n = normalization_X(X_test)
y_train_n = normalization_X(y_train)
y_test_n = normalization_X(y_test)

In [11]:
# Cross validation
num_folds = 5
lambda_choices = [pow(10, -3+0.1*i) for i in range(61)]

X_train_folds = []
y_train_folds = []

X_train_folds = np.array_split(X_train_n, num_folds)
y_train_folds = np.array_split(y_train_n, num_folds)

lambda_to_bic = {}

for lamb in lambda_choices:
    bic = []
    for i in range(num_folds):
        Xtr = np.concatenate(X_train_folds[:i] + X_train_folds[i+1:])
        ytr = np.concatenate(y_train_folds[:i] + y_train_folds[i+1:])
        Xcv = X_train_folds[i]
        ycv = y_train_folds[i]
        beta = lasso_proximal(Xtr,ytr,lamb,MAX_ITER=10000)
        ycv_predict=Xcv.dot(beta)
        bic_predict = get_BIC(ycv,ycv_predict,beta)
        bic.append(bic_predict)
    lambda_to_bic[lamb] = bic
        
i = 0  
for lamb in sorted(lambda_to_bic):
    bic = np.mean(lambda_to_bic[lamb])
    print('%d, lambda = %f, BIC = %f' % (i, lamb, bic))
    i = i + 1

0, lambda = 0.001000, BIC = -35.274220
1, lambda = 0.001259, BIC = -35.274252
2, lambda = 0.001585, BIC = -35.274165
3, lambda = 0.001995, BIC = -35.274110
4, lambda = 0.002512, BIC = -35.274217
5, lambda = 0.003162, BIC = -35.274295
6, lambda = 0.003981, BIC = -35.274393
7, lambda = 0.005012, BIC = -35.274381
8, lambda = 0.006310, BIC = -35.274580
9, lambda = 0.007943, BIC = -35.274668
10, lambda = 0.010000, BIC = -35.274776
11, lambda = 0.012589, BIC = -35.274985
12, lambda = 0.015849, BIC = -35.275170
13, lambda = 0.019953, BIC = -35.275488
14, lambda = 0.025119, BIC = -35.275902
15, lambda = 0.031623, BIC = -35.276223
16, lambda = 0.039811, BIC = -35.276711
17, lambda = 0.050119, BIC = -35.277250
18, lambda = 0.063096, BIC = -35.277756
19, lambda = 0.079433, BIC = -35.278478
20, lambda = 0.100000, BIC = -35.278968
21, lambda = 0.125893, BIC = -35.279346
22, lambda = 0.158489, BIC = -35.279353
23, lambda = 0.199526, BIC = -35.278555
24, lambda = 0.251189, BIC = -35.276421
25, lambda

In [18]:
# According to BIC, we know when lambda = 39.810717, the model works best
beta = lasso_proximal(X_train_n,y_train_n,39.810717,MAX_ITER=10000)
# Output parameter matrix
beta

array([[-0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.        ],
       [-0.        ],
       [ 0.2842712 ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.00433538],
       [-0.15349921],
       [ 0.02196104],
       [-0.39434989]])

In [14]:
y_test_predict=X_test_n.dot(beta)

# Model evaluation (BIC and mse)
bic_predict = get_BIC(y_test_n,y_test_predict,beta)
mse_final = mean_squared_error(y_test_n,y_test_predict)
print(bic_predict)

# Although using BIC to choose model is good enough, MSE is still an important evaluation criteria
print(mse_final)

[-120.63166839]
0.33584819990680576


In [22]:
boston['feature_names_Chinese'] = np.array(['城市人均犯罪率',
                                            '占地面积超过 25,000 平方英尺的住宅用地比例',
                                            '每个城镇非零售业务的比例',
                                            'Charles River 虚拟变量（如果是河道，则为 1; 否则为 0）',
                                            '一氧化氮浓度（每千万份）',
                                            '每间住宅的平均房间数',
                                            '1940 年以前建造的自住单位比例',
                                            '加权距离波士顿的五个就业中心',
                                            '径向高速公路的可达性指数', 
                                            '每 10000 美元的全值财产税率',
                                            '城镇的学生与教师比例',
                                            'B=1000(Bk −0.63)2 其中 Bk 是城镇黑人的比例',
                                            '人口中低收入者的比例'])
for i in range(len(beta)):
    print(round(beta[i,0],4), boston['feature_names'][i], boston['feature_names_Chinese'][i])

-0.0 CRIM 城市人均犯罪率
0.0 ZN 占地面积超过 25,000 平方英尺的住宅用地比例
-0.0 INDUS 每个城镇非零售业务的比例
0.0 CHAS Charles River 虚拟变量（如果是河道，则为 1; 否则为 0）
-0.0 NOX 一氧化氮浓度（每千万份）
0.2843 RM 每间住宅的平均房间数
-0.0 AGE 1940 年以前建造的自住单位比例
-0.0 DIS 加权距离波士顿的五个就业中心
-0.0 RAD 径向高速公路的可达性指数
-0.0043 TAX 每 10000 美元的全值财产税率
-0.1535 PTRATIO 城镇的学生与教师比例
0.022 B B=1000(Bk −0.63)2 其中 Bk 是城镇黑人的比例
-0.3943 LSTAT 人口中低收入者的比例


### 根据参数的估计结果可以得到，对于房价预测较为有效的变量有每间住宅的平均房间数，每 10000 美元的全值财产税率，城镇的学生与教师比例，B=1000(Bk −0.63)2 其中 Bk 是城镇黑人的比例（这一变量与黑人比例正相关）和人口中低收入者的比例。
### 其中每间住宅的平均房间数，城镇的学生与教师比例，人口中低收入者的比例，这三个变量的系数绝对值较大，在房价预测的过程中较为重要。
### 与房价正相关的是每间住宅的平均房间数和黑人比例。前者容易解释，房间越多通常意味着房子更大，而大房子理所当然卖的比较贵。城镇中的黑人比例与房价正相关则可能是商家不希望有大量黑人住进自己出售的小区中（这可能会让有购买力的白人选择其他商家的小区），因而通过提价的方式筛选消费者。但这一点并不显著，可以看到这一项的系数的绝对值非常小。
### 每 10000 美元的全值财产税率，城镇的学生与教师比例，人口中低收入者的比例，这三个变量与房价负相关。这些都比较好理解，税率高的地方人们购买力低，学生的购买力低，所以学生比例高的地方群体的购买力也比较低，低收入者比例高的群体购买力也不高，而房价与人们的购买力正相关，因此与这三个变量负相关。

In [15]:
# Show the differences between the prediction and the real data
comparison = ['predict: '+str(a)+' realcat: '+str(b) for a,b in zip(y_test_predict,y_test_n)]
for comp in comparison:
    print(comp)

predict: [0.68104429] realcat: 0.24780210802208244
predict: [-0.66318347] realcat: -0.21707260770315176
predict: [0.21246598] realcat: -0.23920949892816282
predict: [-0.19508234] realcat: -0.11745659719060171
predict: [0.24330481] realcat: 0.048570086996981954
predict: [1.37715847] realcat: 2.350806774398142
predict: [-2.0258569] realcat: -1.3571225057912264
predict: [0.40040193] realcat: 0.41382879220966606
predict: [0.21719516] realcat: 0.03750164138447661
predict: [0.02462065] realcat: -0.19493571647814067
predict: [-0.0828477] realcat: -0.46057841117827436
predict: [-0.05034548] realcat: -0.25027794454066854
predict: [-0.31784193] realcat: -0.2613463901531743
predict: [0.45803775] realcat: 0.004296304546959799
predict: [-0.69696995] realcat: -1.7223812110039105
predict: [-0.83770832] realcat: -1.390327842628743
predict: [-0.08670846] realcat: -0.1727988252531296
predict: [-0.81435517] realcat: -0.8701108988409809
predict: [-0.18699752] realcat: -0.34989395505321863
predict: [1.2146

## 下面是使用sklearn中的lasso，用来与上面自己写的lasso进行效果对比

In [16]:
# Use lasso which is in sklearn for comparison
# According to MSE and the approximate value of parameters, 
# we can see that the program I wrote is similar to the one in sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
lasso = linear_model.Lasso(alpha = 0.1)
lasso.fit(X_train_n,y_train_n)
y_predict = lasso.predict(X_test_n)
comparison = ['predict: '+str(a)+' realcat: '+str(b) for a,b in zip(y_predict,y_test_n)]
for comp in comparison:
    print(comp)
mse = mean_squared_error(y_test_n,lasso.predict(X_test_n))
score = lasso.score(X_test_n,y_test_n)
print('Score:{}'.format(score))
print('MSE:{}'.format(mse))

predict: 0.6708499096392414 realcat: 0.24780210802208244
predict: -0.5625818307603626 realcat: -0.21707260770315176
predict: 0.11445727684021946 realcat: -0.23920949892816282
predict: -0.5338705211839038 realcat: -0.11745659719060171
predict: 0.15912254554955207 realcat: 0.048570086996981954
predict: 1.2638185533120756 realcat: 2.350806774398142
predict: -1.8678112851845607 realcat: -1.3571225057912264
predict: 0.33588013212199413 realcat: 0.41382879220966606
predict: 0.16903676016794558 realcat: 0.03750164138447661
predict: -0.001266149528744344 realcat: -0.19493571647814067
predict: 0.13266981330501093 realcat: -0.46057841117827436
predict: 0.060879910103654934 realcat: -0.25027794454066854
predict: -0.3588358925070914 realcat: -0.2613463901531743
predict: 0.3573295726344921 realcat: 0.004296304546959799
predict: -0.640831472202977 realcat: -1.7223812110039105
predict: -0.7880749666048401 realcat: -1.390327842628743
predict: -0.016033200957517323 realcat: -0.1727988252531296
predict:

In [17]:
lasso.coef_
# lasso.intercept_

array([-0.        ,  0.        , -0.        ,  0.00956165, -0.        ,
        0.28910624, -0.        , -0.        , -0.        , -0.00620989,
       -0.15944122,  0.02927934, -0.39785725])